In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import annotations

import numpy as np

from mhsxtrapy.field2d import Field2dData, check_fluxbalance, FluxBalanceState
from mhsxtrapy.vis import (
    plot_magnetogram_2D,
    plot_magnetogram_3D,
    plot_dpressure_z,
    plot_ddensity_z,
    plot_dpressure_xy,
    plot_ddensity_xy,
)
from mhsxtrapy.field3d import calculate_magfield
from mhsxtrapy.b3d import WhichSolution
from mhsxtrapy.examples import multipole

In [ ]:
"""
EXAMPLE FOR MAGNETIC FIELD EXTRAPOLATION FROM ANALYTICAL BOUNDARY CONDITION INSTANTIATED "BY HAND" AND FOR ALL THREE DIFFERENT SOLUTIONS
"""

In [ ]:
"""
INSTANTIATE BOUNDARY CONDITION 1

We instantiate the boundary condition manually from the analytical expression given in the following cell using the function multipole(x, y).

We choose a grid resolution of 200 in x- and y-direction (and resulting number of Fourier modes = 200) as well as resolution in z-direction of 400.
We want the boundary condition to extend to 20L (= 20 Mm) in all directions. Therefore, the extrapolation will lead to a 3D data cube.
"""

nx, ny, nz, nf = 200, 200, 400, 200
xmin, xmax, ymin, ymax, zmin, zmax = 0.0, 20.0, 0.0, 20.0, 0.0, 20.0

"""
Calculation of pixel sizes and arrays of x-, y- and z-extension of box. 
"""
pixelsize_x = (xmax - xmin) / nx
pixelsize_y = (ymax - ymin) / ny
pixelsize_z = (zmax - zmin) / nz

x_arr = np.linspace(xmin, xmax, nx, dtype=np.float64)
y_arr = np.linspace(ymin, ymax, ny, dtype=np.float64)
z_arr = np.linspace(zmin, zmax, nz, dtype=np.float64)

In [ ]:
"""
INSTANTIATE BOUNDARY CONDITION 2

We define the normalising magnetic field strength as 500 Gauss. 
"""

B_PHOTO = 500

data_bz = np.zeros((ny, nx))

for ix in range(0, nx):
    for iy in range(0, ny):
        x = x_arr[ix] / 10.0
        y = y_arr[iy] / 10.0
        data_bz[iy, ix] = multipole(x, y)

In [ ]:
"""
CHECK FLUXBALANCE OF BOUNDARY CONDITION

check_fluxbalance returns the summation of the flux through the bottom boundary, normalised by the sum of absolute values. 
This returns a value between −1 and 1, where either extreme corresponds to the flux being directed entirely inward or entirely outward, respectively. We consider values 
between −0.01 and 0.01 as flux-balanced in agreement with Wiegelmann & Sakurai (2021). 
"""

np.fabs(check_fluxbalance(data_bz)) < 0.01

In [ ]:
"""
INSTANTIATE Field2dData OBJECT FROM ANALYTICAL BOUNDARY CONDITION
"""

data2d = Field2dData(
    nx,
    ny,
    nz,
    nf,
    pixelsize_x,
    pixelsize_y,
    pixelsize_z,
    x_arr,
    y_arr,
    z_arr,
    data_bz,
    flux_balance_state=FluxBalanceState.UNBALANCED,
)

In [ ]:
"""
PLOT 2D BOUNDARY CONDITION
"""

plot_magnetogram_2D(data2d)

In [ ]:
"""
EXTRAPOLATE MAGNETIC FIELD 1

Use asymptotic solution presented in Nadol and Neukirch (2025) and choosen parameters alpha, a, b, z0 and delta z. 
"""

data3d = calculate_magfield(
    data2d,
    alpha=0.05,
    a=0.22,
    which_solution=WhichSolution.ASYMP,
    b=1.0,
    z0=2.0,
    deltaz=0.2,
)

In [ ]:
"""
EXTRAPOLATE MAGNETIC FIELD 2

Use asymptotic solution presented in Neukirch and Wiegelmann (2019) and choosen parameters alpha, a, b, z0 and delta z. 
"""

data3d_NW = calculate_magfield(
    data2d,
    alpha=0.05,
    a=0.22,
    which_solution=WhichSolution.NEUWIE,
    b=1.0,
    z0=2.0,
    deltaz=0.2,
)

In [ ]:
"""
EXTRAPOLATE MAGNETIC FIELD 3

Use asymptotic solution presented in Low (1990, 1991) and choosen parameters alpha and kappa. 
"""

data3d_LOW = calculate_magfield(
    data2d,
    alpha=0.05,
    a=0.22,
    kappa=1/2.0,
    which_solution=WhichSolution.LOW,
    b=1.0,
    z0=2.0,
    deltaz=0.2,
)

In [ ]:
"""
PLOT MAGNETOGRAM
"""

plot_magnetogram_3D(data3d, view="los", footpoints="active-regions")

In [ ]:
"""
PLOT PRESSURE VARIATION WITH HEIGHT z
"""

plot_dpressure_z(data3d)

In [ ]:
"""
PLOT DENSITY VARIATION WITH HEIGHT z
"""

plot_ddensity_z(data3d)

In [ ]:
"""
PLOT PRESSURE VARIATION FOR A SPECIFIC HEIGHT z [Mm] FOR ALL x and y
"""

plot_dpressure_xy(data3d, z=0) # type: ignore

In [ ]:
"""
PLOT DENSITY VARIATION FOR A SPECIFIC HEIGHT z [Mm] FOR ALL x and y
"""

plot_ddensity_xy(data3d, z=0) # type: ignore